<h1>**IBM Model 1**</h1>

1. a) Implement EM training (Brown et al., 1993) for IBM model 1; <br />
    b) Implement variational inference for Bayesian IBM model 1; <br />
    c) All of the tasks below should be performed for both models.<br />
2. Plot the evolution of training log likelihood (or ELBO) as a function of the iteration.
3. Plot the evolution of alignment error rate (AER) on validation data as a function of the iteration;
4. Experiment with two criteria for model selection (i.e. deciding on number of training iterations): 
    1) convergence in terms of training log likelihood; 
    2) best AER on validation data;
5. For the selected models, obtain Viterbi alignments for every sentence pair in a test
corpus and compute AER using a gold-standard provided by the assistant;

In [ ]:
import aer
from collections import defaultdict, Counter
from math import log2
import numpy as np
from random import randint
import progressbar


def read_corpus(file_name, source_language):
    """
    Reads the corpus and saves each sentence in a list.
    """
    
    corpus = []
    
    with open(file_name, "r") as f:
        for line in f:
            line = line.replace("\n", "")
            sentence = line.split()
            
            if source_language:
                sentence.insert(0, "null")
            corpus.append(sentence)
    return corpus[:2000]


def reduce_corpus(corpus):
    flat_corpus = [word for sentence in corpus for word in sentence]
    word_counts = Counter(flat_corpus)
    small_corpus = []
    
    for sentence in corpus:
        small_sentence = []
        
        for word in sentence:
            if word_counts[word] != 1:
                small_sentence.append(word)
            else:
                small_sentence.append("-LOW-")
        small_corpus.append(small_sentence)
    return small_corpus
    

def initialise_parameters(source_corpus, target_corpus):
    """
    Initialises the conditional probability of generating a source 
    word from a target word for all possible pairs of words in the source 
    and target sentences to 5 and then normalises the parameters such that 
    the initialisation is uniform.
    """
    
    flat_corpus = [word for sentence in source_corpus for word in sentence]
    amount_source_words = len(set(flat_corpus))
    theta0 = 1/amount_source_words
    parameters = defaultdict(lambda: defaultdict(lambda: theta0))
    
#     with progressbar.ProgressBar(max_value=len(target_corpus)) as bar:
#         for n in range(len(target_corpus)):
#             target_sentence = target_corpus[n]
#             source_sentence = source_corpus[n]
            
#             for i in range(len(target_sentence)):
#                 target_word = target_sentence[i]
                
#                 for j in range(len(source_sentence)):        
#                     source_word = source_sentence[j]

#                     if source_word not in parameters:
#                         parameters[source_word] = {target_word: theta0}
#                     elif target_word not in parameters[source_word]:
#                         target_words = parameters[source_word]
#                         target_words[target_word] = theta0
#                         parameters[source_word] = target_words
#             bar.update(n)
                    
#     # Normalise parameters
#     for source_word, target_words in parameters.items():
#         sum_target_words = sum(target_words.values())
#         for target_word, value in target_words.items():
#             parameters[source_word][target_word] = value/sum_target_words
    return parameters


def expectation_maximisation(source_corpus, target_corpus, initial_params, 
                             num_iterations, epsilon, min_perplexity_change):
    """
    Do the EM algorithm until perplexity decreases very little or until 
    the number of iterations is reached.
    """
    
    parameters = initial_params
    old_perplexity = -100000
    
    for k in range(0, num_iterations):
        print("Iteration #" + str(k), "out of", num_iterations - 1)
        counts_single = defaultdict(lambda: 1.0)
        counts_pairs = defaultdict(lambda: defaultdict(float))
        counts_single, counts_pairs = e_step(source_corpus, target_corpus,
                                             parameters, counts_single, 
                                             counts_pairs)
        parameters = m_step(parameters, counts_single, counts_pairs)
        perplexity = compute_perplexity(parameters, source_corpus, target_corpus)
        print("perplexity:", perplexity)
        
        if abs(perplexity - old_perplexity) < min_perplexity_change:
            return parameters
        else:
            old_perplexity = perplexity
    return parameters
    
    
def e_step(source_corpus, target_corpus, parameters, counts_single, 
           counts_pairs):
    """
    Do the E-step by computing the expected counts.
    """
    
    print("Doing E-step...")
    
    with progressbar.ProgressBar(max_value=len(target_corpus)) as bar:
        for n in range(len(target_corpus)):
            target_sentence = target_corpus[n]
            source_sentence = source_corpus[n]

            for i in range(len(target_sentence)):
                normalisation_term = 0
                target_word = target_sentence[i]

                for j in range(len(source_sentence)):
                    source_word = source_sentence[j]
                    normalisation_term += parameters[source_word][target_word]
                for j in range(len(source_sentence)):
                    source_word = source_sentence[j]
                    expected_count = parameters[source_word][target_word]/normalisation_term
                    counts_pairs[source_word][target_word] += expected_count
                    counts_single[source_word] += expected_count
            bar.update(n)
    return counts_single, counts_pairs


def m_step(parameters, counts_single, counts_pairs):
    """
    Do the M-step by normalising the parameters.
    """
    
    print("Doing M-step...")
    
    with progressbar.ProgressBar(max_value=len(counts_pairs)) as bar:
        i = 0
        for source_word, target_words in counts_pairs.items():
            for target_word, expected_count in target_words.items():
                parameters[source_word][target_word] = expected_count/counts_single[source_word]
            i += 1
            bar.update(i)
    return parameters


def compute_perplexity(theta_dict, source_corpus, target_corpus):
    logprobs = []
    total_sum = 0
    
    for n in range(len(source_corpus)):
        english_sentence = source_corpus[n]
        french_sentence = target_corpus[n]
        french_sum = 0
        for j in range(len(french_sentence)): 
            f_j = french_sentence[j]
            log_sum = []
            for i in range(len(english_sentence)): 
                e_i = english_sentence[i]
                log_sum.append(theta_dict[e_i][f_j])
            french_sum += np.log(np.sum(log_sum))
        total_sum += french_sum
    perplexity = total_sum
    return perplexity


def calculate_likelihood(source_corpus, target_corpus, parameters):
    """
    Calculates the likelihood over te corpus:
    sum_<f,e> sum^m_i=1 log sum^l_j=0 pi_t(target|source) 
    """
    
    corpus_likelihood = 0
    
    for n in range(len(source_corpus)):
        source_sentence = source_corpus[n]
        target_sentence = target_corpus[n]
        target_likelihood = 0
        
        for i in range(len(target_sentence)):
            target_word = target_sentence[i]
            source_likelihood = 0
            
            for j in range(len(source_sentence)):
                source_word = source_sentence[j]
                source_likelihood += parameters[source_word][target_word]
            target_likelihood += log2(source_likelihood)
        corpus_likelihood += target_likelihood
    return corpus_likelihood
   
    
def get_best_alignment(source_corpus, target_corpus, parameters):
    """
    Gets the best alignment for each sentence and saves the alignment
    in a list of lists that holds tuples for each position in the sentence
    and looks as follows:
    (sentence_index, target_word_index, source_word_index).
    """
    
    alignments = []
    
    for n in range(len(source_corpus)):
        source_sentence = source_corpus[n]
        target_sentence = target_corpus[n]
        alignment = []
        
        for i in range(len(target_sentence)):
            target_word = target_sentence[i]
            best_prob = 0
            best_j = 0
            
            for j in range(len(source_sentence)):
                source_word = source_sentence[j]
                prob = parameters[source_word][target_word]
                
                if prob > best_prob:
                    best_prob = prob
                    best_j = j
                    
            if best_j != 0:    
                alignment.append((n, best_j, i+1))
        alignments.append(alignment)
    return alignments


def compute_aer(predictions):
    gold_sets = aer.read_naacl_alignments("validation/dev.wa.nonullalign")
    metric = aer.AERSufficientStatistics()
    
    for gold, prediction in zip(gold_sets, predictions):
        prediction = set([(alignment[1], alignment[2]) for alignment in prediction])
        metric.update(sure=gold[0], probable=gold[1], predicted=prediction)
    print(metric.aer())
    
    
train_source_corpus = read_corpus("training/hansards.36.2.e", True)
train_source_corpus = reduce_corpus(train_source_corpus)
print("Read training source corpus")
train_target_corpus = read_corpus("training/hansards.36.2.f", False)
train_target_corpus = reduce_corpus(train_target_corpus)
print("Read training target corpus")
val_target_corpus = read_corpus("validation/dev.f", False)
val_source_corpus = read_corpus("validation/dev.e", True)

print("Initialising parameters...")
initial_params = initialise_parameters(train_source_corpus, train_target_corpus)
parameters = expectation_maximisation(train_source_corpus, train_target_corpus, 
                                      initial_params, 1, 1, 5)
alignments = get_best_alignment(val_source_corpus, val_target_corpus, parameters)
compute_aer(alignments)